# Deribit Api Management

## Api configuration

In [1]:
import pprint
import nest_asyncio
import asyncio
import websockets
import json
import pandas as pd
import datetime as dt
import time

nest_asyncio.apply()

async def call_api(msg):
   async with websockets.connect('wss://test.deribit.com/ws/api/v2') as websocket:
       await websocket.send(msg)
       while websocket.open:
           response = await websocket.recv()
           return response

def async_loop(api, message):
    return asyncio.get_event_loop().run_until_complete(api(message))


## Simple api calls (getting information)

In [2]:
def retrieve_historic_data(start, end, instrument, timeframe = '1'):
    msg = \
        {
            "jsonrpc": "2.0",
            "id": 833,
            "method": "public/get_tradingview_chart_data",
            "params": {
                "instrument_name": instrument,
                "start_timestamp": start,
                "end_timestamp": end,
                "resolution": timeframe
            }
        }
    resp = async_loop(call_api, json.dumps(msg))

    return resp

def getSummary (currency = 'BTC', type = 'option'):
    msg = \
    {
        "jsonrpc" : "2.0",
        "id" : 9344,
        "method" : "public/get_book_summary_by_currency",
        "params" : {
            "currency" : currency,
            "kind" : type
        }
    }
    resp = async_loop(call_api, json.dumps(msg))

    return resp

def epochToDate(epoch):
    return time.strftime('%d-%m-%Y %H:%M',time.localtime(epoch/1000))


instruments_json_resp = getSummary()


## Cleaning data

In [3]:
def historyCleaning(json_resp):
    res = json.loads(json_resp)

    df = pd.DataFrame(res['result'])

    df['ticks'] = df.ticks / 1000
    df['timestamp'] = [dt.datetime.fromtimestamp(date) for date in df.ticks]

    return df

def instrumentsCleaning(json_resp):
    res = json.loads(json_resp)

    df = pd.DataFrame(res['result']).sort_values(["instrument_name"]) 

    # df['expiration_timestamp'] = df['expiration_timestamp'].apply(epochToDate)

    # df.drop(['base_currency','creation_timestamp','kind','option_type','quote_currency','min_trade_amount','is_active','block_trade_commission'],axis = 1 , inplace=True)
    df = df[['instrument_name','mark_price','volume','estimated_delivery_price','creation_timestamp']]

    return df

instrument_df = instrumentsCleaning(instruments_json_resp)
instrument_df.head()


,instrument_name,mark_price,volume,estimated_delivery_price,creation_timestamp
221,BTC-14FEB22-39000-C,0.085163,0.0,42629.29,1644764710509
112,BTC-14FEB22-39000-P,0.000213,30.0,42629.29,1644764710507
186,BTC-14FEB22-40000-C,0.062209,0.0,42629.29,1644764710508
62,BTC-14FEB22-40000-P,0.000722,224.9,42629.29,1644764710506
2,BTC-14FEB22-41000-C,0.039255,0.0,42629.29,1644764710505


## Separating Calls and Puts

In [4]:
instrument_df['optionType'] = [j[-1] for j in instrument_df['instrument_name']]

put_df = instrument_df[instrument_df['optionType'] == 'P'].drop('optionType',axis=1)
call_df = instrument_df[instrument_df['optionType'] == 'C'].drop('optionType',axis=1)


# put_df = put_df.set_index('instrument_name')
# call_df = call_df.set_index('instrument_name')

# put_df.set_index('instrument_name', inplace=True)
# call_df.set_index('instrument_name', inplace=True)



print(call_df.head())
print(put_df.head())


         instrument_name  mark_price  volume  estimated_delivery_price  \
221  BTC-14FEB22-39000-C    0.085163     0.0                  42629.29   
186  BTC-14FEB22-40000-C    0.062209     0.0                  42629.29   
2    BTC-14FEB22-41000-C    0.039255     0.0                  42629.29   
106  BTC-14FEB22-42000-C    0.018044  1592.9                  42629.29   
85   BTC-14FEB22-43000-C    0.004425   473.5                  42629.29   

     creation_timestamp  
221       1644764710509  
186       1644764710508  
2         1644764710505  
106       1644764710507  
85        1644764710506  
         instrument_name  mark_price  volume  estimated_delivery_price  \
112  BTC-14FEB22-39000-P    0.000213    30.0                  42629.29   
62   BTC-14FEB22-40000-P    0.000722   224.9                  42629.29   
217  BTC-14FEB22-41000-P    0.001230  1262.5                  42629.29   
84   BTC-14FEB22-42000-P    0.003482   788.6                  42629.29   
60   BTC-14FEB22-43000-P    0

## Eporting to Excel

In [5]:
file_name = 'Prueba.xlsx'

put_df.to_excel('/Users/jvera/My Drive/codingProjects/deribit/' + file_name)

## Creating mark price graph

### Retrieve and clean information

In [9]:
END_TIME = round(time.time() * 1000) 
# End time is actual time

start_time = int(put_df['creation_timestamp'].iloc[0])

instrument = put_df['instrument_name'].iloc[0]




historic_json_resp = retrieve_historic_data(start_time, END_TIME, instrument, "1")

historic_df = historyCleaning(historic_json_resp)
historic_df.tail()

,volume,ticks,status,open,low,high,cost,close,timestamp
0,0.0,1.644765e+09,ok,0.0015,0.0015,0.0015,0.0,0.0015,2022-02-13 12:05:00


### Plotting

In [7]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(x=historic_df.index,y=historic_df['close'], mode='lines'))


In [8]:
fig.show()